Blog here: https://wordpress.com/post/brianfarish3.wordpress.com/168

We are here: https://docs.aws.amazon.com/textract/latest/dg/api-async.html

AWS recommends using an asynchronous function to process multiple page files. So we're looking at how to do that! 